In [6]:
%matplotlib 

import json
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider
import numpy as np

# Load the JSON data
with open('../data/agent_logs.json', 'r') as f:
    raw_data = json.load(f)

# Combine all generations into a single dictionary
data = {k: v for entry in raw_data for k, v in entry.items()}
generations = list(data.keys())
num_generations = len(generations)

# Create a fixed-position layout for the nodes
G_fixed = nx.Graph()
nodes = set()
edges_by_generation = []

for gen in generations:
    agents_data = data[gen]  # Access agent data for the generation
    edges = []
    for agent_id, agent_info in agents_data.items():
        nodes.add(agent_id)
        for _, upvoted_agent in agent_info.get("upvoted_messages", []):
            edges.append((agent_id, str(upvoted_agent)))  # Ensure both are strings
    edges_by_generation.append(edges)

G_fixed.add_nodes_from(nodes)
pos = nx.spring_layout(G_fixed, seed=42)  # Fixed layout

# Create the figure and axis
fig, ax = plt.subplots(figsize=(10, 8))
plt.subplots_adjust(bottom=0.15)

def draw_network(gen_idx):
    ax.clear()
    G = nx.Graph()
    G.add_nodes_from(nodes)
    G.add_edges_from(edges_by_generation[gen_idx])
    nx.draw(G, pos, with_labels=True, node_size=500, node_color='skyblue', edge_color='gray', font_size=8, ax=ax)
    ax.set_title(f'Generation {gen_idx + 1}')
    fig.canvas.draw_idle()  # Ensure updates

# Initial draw
draw_network(0)

# Add slider
ax_slider = plt.axes([0.2, 0.02, 0.65, 0.03])
slider = Slider(ax_slider, 'Generation', 1, num_generations, valinit=1, valstep=1)

def update(val):
    gen_idx = int(slider.val) - 1
    draw_network(gen_idx)

slider.on_changed(update)

plt.show()

ImportError: Failed to import any of the following Qt binding modules: PyQt6, PySide6, PyQt5, PySide2